In [1]:
import urllib.request
import pandas as pd
from datetime import datetime
import os
import glob
from bs4 import BeautifulSoup

In [2]:
def finde_files(pattern):
    files = glob.glob(pattern)
    return files

def delete_files(pattern):
    files = finde_files(pattern)
    for file in files:
        os.remove(file)

In [3]:
for i in range(1, 28):
    file_pattern = f"vhi_id_{i}_*"
    delete_files(file_pattern)
    url=f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    time = datetime.now()
    now = time.strftime("%d_%m_%Y_%H;%M;%S")
    out = open(f'vhi_id_{i}_downloaded_time{now}.csv','wb')
    out.write(vhi_url.read()) 
    out.close()
    print(f"VHI {i} is downloaded...")

VHI 1 is downloaded...
VHI 2 is downloaded...
VHI 3 is downloaded...
VHI 4 is downloaded...
VHI 5 is downloaded...
VHI 6 is downloaded...
VHI 7 is downloaded...
VHI 8 is downloaded...
VHI 9 is downloaded...
VHI 10 is downloaded...
VHI 11 is downloaded...
VHI 12 is downloaded...
VHI 13 is downloaded...
VHI 14 is downloaded...
VHI 15 is downloaded...
VHI 16 is downloaded...
VHI 17 is downloaded...
VHI 18 is downloaded...
VHI 19 is downloaded...
VHI 20 is downloaded...
VHI 21 is downloaded...
VHI 22 is downloaded...
VHI 23 is downloaded...
VHI 24 is downloaded...
VHI 25 is downloaded...
VHI 26 is downloaded...
VHI 27 is downloaded...


In [4]:
for i in range(1, 28):
    file_pattern = f"vhi_id_{i}_*"
    file_path = finde_files(file_pattern)[0]

    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
    soup = BeautifulSoup(file_content, 'html.parser')
    cleaned_text = soup.get_text()
    with open(file_path, 'w', encoding='utf-8') as temp_file:
        temp_file.write(cleaned_text)

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = [line.rstrip(',\n') for line in file]
    with open(file_path, 'w', encoding='utf-8') as temp_file:
        temp_file.write('\n'.join(lines))

    data = pd.read_csv(file_path, lineterminator ='\n', skiprows=1)
    data.columns = [col.replace('\r', '') for col in data.columns]
    data.columns = [col.replace(' ', '') for col in data.columns]
    data['area'] = i
    data = data.drop(data.loc[data['VHI'] == -1].index)
    delete_files(file_pattern)
    data.to_csv(file_path, index=False, encoding='utf-8')

In [5]:
for i in range(1, 28):
    file_pattern = f"vhi_id_{i}_*"
    file_path = finde_files(file_pattern)[0]
    data = pd.read_csv(file_path)
    area_index = {
        1: 'Vinnytsya', 2: 'Volyn', 3: "Dnipropetrovs'k", 4: "Donets'k", 5: 'Zhytomyr', 6: 'Transcarpathia', 7: 'Zaporizhzhya', 8: "Ivano-Frankivs'k", 9: 'Kiev City',
        10: 'Kiev', 11: 'Kirovohrad', 12: "Luhans'k", 13: "L'viv", 14: 'Mykolayiv', 15: 'Odessa', 16: 'Poltava', 17: 'Rivne', 18: "Sevastopol'",
        19: 'Sumy', 20: "Ternopil'", 21: 'Kharkiv', 22: 'Kherson', 23: "Khmel'nyts'kyy", 24: 'Cherkasy', 25: 'Chernivtsi', 26: 'Chernihiv', 27: 'Crimea',
    }
    data['area'] = data['area'].replace(area_index)
    data.to_csv(f'mod_{file_path}', index=False, encoding='utf-8')

In [6]:
data_frames = []

for i in range(1, 28):
    file_pattern = f"mod_vhi_id_{i}_*"
    file_path = finde_files(file_pattern)[0]
    data = pd.read_csv(file_path)
    data_frames.append(data)
    
data = pd.concat(data_frames)

data.to_csv('vhi_end.csv', index=False, encoding='utf-8')

In [7]:
def vhi(region, year):
    data = pd.read_csv('vhi_end.csv')
    VHIs = data[(data["area"] == region) & (data["year"] == year)]['VHI']
    VHIs.to_csv('vhi_row.csv', index=False, encoding='utf-8')
    return VHIs

data = vhi("Ternopil'", 2017)
print(data)

43304    54.10
43305    53.72
43306    52.13
43307    49.95
43308    50.07
43309    48.82
43310    47.34
43311    46.10
43312    45.34
43313    44.11
43314    42.67
43315    43.09
43316    44.58
43317    45.97
43318    47.94
43319    50.14
43320    56.26
43321    62.28
43322    64.26
43323    67.14
43324    67.63
43325    68.67
43326    68.37
43327    66.86
43328    66.32
43329    66.09
43330    64.63
43331    63.37
43332    62.34
43333    60.25
43334    59.06
43335    59.11
43336    58.67
43337    57.50
43338    53.19
43339    49.54
43340    48.21
43341    47.39
43342    48.85
43343    50.30
43344    52.69
43345    53.94
43346    51.98
43347    46.19
43348    46.21
43349    42.56
43350    39.65
43351    37.05
43352    36.24
43353    36.14
43354    38.05
43355    41.78
Name: VHI, dtype: float64


In [8]:
def data_filter(regions, years):
    data = pd.read_csv('vhi_end.csv')
    filtered_data = data[(data['area'].isin(regions)) & (data['year'].isin(years))]
    filtered_data.to_csv('vhi_row.csv', index=False, encoding='utf-8')
    return filtered_data

def minmax(regions, years):
    data = data_filter(regions, years)
    min_v = data['VHI'].min()
    max_v = data['VHI'].max()
    avrg = data['VHI'].mean()
    mediana = data['VHI'].median()
    return min_v, max_v, round(avrg, 2), mediana

minimum, maximum, avrg, mediana = minmax(['Poltava', 'Kiev'], [2014, 2015])
print(f"Мінімальне VHI: {minimum}")
print(f"Максимальне VHI: {maximum}")
print(f"Середнє VHI: {avrg}")
print(f"Медіана VHI: {mediana}")

Мінімальне VHI: 19.95
Максимальне VHI: 64.33
Середнє VHI: 43.3
Медіана VHI: 42.685


In [9]:
def vhi_years(fromy, toy, regions):
    data = pd.read_csv('vhi_end.csv')
    years = list(range(fromy, toy+1))
    vhi = data[(data['area'].isin(regions)) & (data['year'].isin(years))]
    vhi.to_csv('vhi_row.csv', index=False, encoding='utf-8')
    return vhi['VHI']

vhi = vhi_years(2011, 2014, ['Poltava', 'Kiev'])
print(vhi)

21132    42.00
21133    39.82
21134    38.35
21135    36.72
21136    35.69
         ...  
34451    37.26
34452    38.34
34453    37.71
34454    37.71
34455    38.44
Name: VHI, Length: 416, dtype: float64


In [32]:
data = pd.read_csv('vhi_end.csv')
df_drought = data[(data.VHI <= 15)]
df_drought = df_drought.drop_duplicates(subset=['year', 'area'])
drought_counts = df_drought.groupby('year')['area'].nunique().reset_index()
extreme_drought = drought_counts[drought_counts['area'] > 5]['year']
extreme_drought_data = df_drought[df_drought['year'].isin(extreme_drought)]
print(extreme_drought_data[['year', 'area', 'VHI']])
extreme_drought_data.to_csv('vhi_rowssss.csv', index=False, encoding='utf-8')

       year              area    VHI
949    2000         Vinnytsya  14.64
16253  2000  Ivano-Frankivs'k  14.61
22809  2000        Kirovohrad  12.51
24993  2000          Luhans'k  14.89
42484  2000         Ternopil'  13.14
51228  2000          Cherkasy  12.26
